### Modelos

- Excecução de cross section
- Efeitos fixos
- Efeitos aleatórios



In [1]:
df = pd.read_parquet('../Dados/pnad_covid/pnad_covid_tratada.parquet')

NameError: name 'pd' is not defined

In [11]:
df.shape

(2508686, 42)

In [1]:
import pandas as pd
import numpy as np

# Modelo
import  statsmodels.api as sm 
from linearmodels.panel import PanelOLS

In [11]:
panel_data = pd.read_parquet('../Dados/pnad_covid/pnad_covid_balanced.parquet')

In [12]:
panel_data.dropna(inplace=True)

In [6]:
panel_data.reset_index(inplace=True)

In [7]:
#for every portfolio, fill mising values with first value of sigla_uf



panel_data['C013'] = panel_data['C013'].fillna(method='bfill')

panel_data['A003'] = panel_data['A003'].fillna(method='bfill')

panel_data['A004'] = panel_data['A004'].fillna(method='bfill')

panel_data['A005'] = panel_data['A005'].fillna(method='bfill')



In [17]:
#for each domicilio_id fill missing values with first value diferent of NaN in column

panel_data['A003'] = panel_data.groupby('domicilio_id')['A003'].transform(lambda x: x.fillna(method='bfill'))

: 

: 

### Regressão

In [8]:
panel_data.set_index(['domicilio_id', 'semana_dt'], inplace=True)

In [9]:
#Variáveis explicativas do modelo
# Adicionando uma constante
exog_variables = panel_data.drop('C013', axis=1)

In [15]:
panel_data

C013    A003    A004   
domicilio_id            semana_dt                        
1100000161Mulher221974  18         Não  Mulher   Parda  \
                        22         Não  Mulher   Parda   
                        26         Não  Mulher   Parda   
                        30         Não  Mulher   Parda   
                        34         Não  Mulher   Parda   
...                                ...     ...     ...   
53001448813Homem2331978 42         Sim   Homem  Branca   
53001448813Mulher721980 42         Não  Mulher  Branca   
5300193734Mulher2081982 44         Sim  Mulher   Parda   
5300317391Homem18112000 44         Sim   Homem   Parda   
5300317391Homem4111997  44         Não   Homem   Parda   

                                                                   A005   
domicilio_id            semana_dt                                         
1100000161Mulher221974  18                               Médio completo  \
                        22                               Médio completo   
                        26                               Médio completo   
                        30                               Médio completo   
                        34                               Médio completo   
...                                                                 ...   
53001448813Homem2331978 42         Pós-graduação, mestrado ou doutorado   
53001448813Mulher721980 42         Pós-graduação, mestrado ou doutorado   
5300193734Mulher2081982 44                            Superior completo   
5300317391Homem18112000 44                          Superior incompleto   
5300317391Homem4111997  44                               Médio completo   

                                  formalidade semana  
domicilio_id            semana_dt                     
1100000161Mulher221974  18             Formal     24  
                        22             Formal     24  
                        26             Formal     24  
                        30             Formal     24  
                        34             Formal     24  
...                                       ...    ...  
53001448813Homem2331978 42             Formal     20  
53001448813Mulher721980 42             Formal     20  
5300193734Mulher2081982 44             Formal     20  
5300317391Homem18112000 44           Informal     20  
5300317391Homem4111997  44           Informal     20  

[977444 rows x 6 columns]

In [14]:
pd.get_dummies(exog_variables[['A003','A004', 'A005', 'formalidade', 'semana']]).drop(['A003_Homem', 'A004_Brancas e Amarelas', 'A005_Sem instrução', 'formalidade_Informal'], axis=1)

KeyError: "['A004_Brancas e Amarelas'] not found in axis"

In [6]:
pd.get_dummies(exog_variables[['A003','A004', 'A005', 'formalidade']])[['A003_Mulher', 'A004_Pretas e Pardas', 'formalidade_Formal', 'A005_Fundamental completo e Médio incompleto', 'A005_Médio completo e Superior incompleto']]

A003_Mulher  A004_Pretas e Pardas   
domicilio_id            semana_dt                                      
1100000161Mulher221974  18                True                  True  \
                        19               False                 False   
                        20               False                 False   
                        21               False                 False   
                        22                True                  True   
...                                        ...                   ...   
5300269062Homem17122010 41               False                 False   
                        42               False                 False   
                        43               False                  True   
                        44               False                 False   
                        45               False                 False   

                                   formalidade_Formal   
domicilio_id            semana_dt                       
1100000161Mulher221974  18                       True  \
                        19                      False   
                        20                      False   
                        21                      False   
                        22                       True   
...                                               ...   
5300269062Homem17122010 41                      False   
                        42                      False   
                        43                      False   
                        44                      False   
                        45                      False   

                                   A005_Fundamental completo e Médio incompleto   
domicilio_id            semana_dt                                                 
1100000161Mulher221974  18                                                False  \
                        19                                                False   
                        20                                                False   
                        21                                                False   
                        22                                                False   
...                                                                         ...   
5300269062Homem17122010 41                                                False   
                        42                                                False   
                        43                                                False   
                        44                                                False   
                        45                                                False   

                                   A005_Médio completo e Superior incompleto  
domicilio_id            semana_dt                                             
1100000161Mulher221974  18                                              True  
                        19                                             False  
                        20                                             False  
                        21                                             False  
                        22                                              True  
...                                                                      ...  
5300269062Homem17122010 41                                             False  
                        42                                             False  
                        43                                             False  
                        44                                             False  
                        45                                             False  

[13080125 rows x 5 columns]

In [7]:
exog_variables = pd.get_dummies(exog_variables).drop(['A003_Homem', 'A004_Brancas e Amarelas', 'A005_Sem instrução', 'formalidade_Informal'], axis=1)

In [8]:
exog_variables.shape

(13080125, 19)

In [9]:
endog_variable = pd.get_dummies(panel_data['C013'])['Sim']

In [10]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Selecionar apenas as variáveis explicativas numéricas
exog_numeric = exog_variables.select_dtypes(include=np.number)

# Calcular os VIFs
vifs = pd.DataFrame()
vifs["Variável"] = exog_numeric.columns
vifs["VIF"] = [variance_inflation_factor(exog_numeric.values, i) for i in range(exog_numeric.shape[1])]

print(vifs)

Empty DataFrame
Columns: [Variável, VIF]
Index: []


In [11]:
# Estimating the fixed effects model
f_effects_model = PanelOLS(dependent = endog_variable,  # Variável endógena (Y)
                           exog = exog_variables,  # Variáveis exógenas (X matrix)
                           check_rank=False,  # Checar se a matriz X é de posto completo
                           entity_effects = True,  # True -> Efeitos fixos
                           drop_absorbed = True,# Remover variáveis absorvidas
                           time_effects=True)  

# Ajustar o modelo aos dados
f_effects_results = f_effects_model.fit()

: 

: 

In [ ]:
f_effects_results = f_effects_model.fit()

In [ ]:
f_effects_results.summary

Dep. Variable:,Sim,R-squared:,0.8433
Estimator:,PanelOLS,R-squared (Between):,0.8934
No. Observations:,1023662,R-squared (Within):,0.8433
Date:,"Sat, Jun 17 2023",R-squared (Overall):,0.8521
Time:,14:03:44,Log-likelihood,8.959e+05
Cov. Estimator:,Unadjusted,,
,,F-statistic:,2.951e+05
Entities:,36558,P-value,0.0000
Avg Obs:,28.001,Distribution:,"F(18,987059)"
Min Obs:,28.000,,
Max Obs:,35.000,F-statistic (robust):,2.951e+05


---

Efeito do 2º choque - Flexibilização